In [1]:
def parse_netlist(netlist):
    global gates
    gates = {}
    fanout_values = {}  # Dictionary to store fanout values for later use
    for line in netlist.split('\n')[1:]:
        if line.strip():  # Skip empty lines
            parts = line.split()
            gate_num = int(parts[0])
            gate_conn = parts[1]
            gate_type = parts[2]
            fanout = int(parts[3])
            fanin = int(parts[4])
            fanin1 = int(parts[5]) if parts[5] != '-' else None
            fanin2 = int(parts[6]) if parts[6] != '-' else None
            if gate_conn == 'fan':
                fanout_values[gate_num] = fanout  # Store fanout value for later use
            
            gates[gate_num] = {'type': gate_type, 'fanout': fanout, 'fanin': fanin, 'fanin1': fanin1, 'fanin2': fanin2}
    
    # Update fanin1 and fanin2 values based on fanout values
    for gate_num, gate in gates.items():
        if gate['type'] != 'from':  # Only update non-'from' gates
            if gate['fanin1'] in fanout_values or gate['fanin2'] in fanout_values:
                if gate['fanin1'] in fanout_values:
                    gates[gate_num]['fanin1'] = fanout_values[gate['fanin1']]
                if gate['fanin2'] in fanout_values:
                    gates[gate_num]['fanin2'] = fanout_values[gate['fanin2']]

    return gates

def generate_boolean_equation(gates):
    def get_boolean_expression(gate_num):
        gate = gates[gate_num]
        if gate['type'] == 'inpt':
            return f'x{gate_num}'
        elif gate['type'] == 'and':
            fanin1_expr = get_boolean_expression(gate['fanin1'])
            fanin2_expr = get_boolean_expression(gate['fanin2'])
            return f'({fanin1_expr} and {fanin2_expr})'
        elif gate['type'] == 'or':
            fanin1_expr = get_boolean_expression(gate['fanin1'])
            fanin2_expr = get_boolean_expression(gate['fanin2'])
            return f'({fanin1_expr} or {fanin2_expr})'
        elif gate['type'] == 'nand':
            fanin1_expr = get_boolean_expression(gate['fanin1'])
            fanin2_expr = get_boolean_expression(gate['fanin2'])
            return f'({fanin1_expr} nand {fanin2_expr})'
        elif gate['type'] == 'nor':
            fanin1_expr = get_boolean_expression(gate['fanin1'])
            fanin2_expr = get_boolean_expression(gate['fanin2'])
            return f'({fanin1_expr} nor {fanin2_expr})'
        elif gate['type'] == 'xor':
            fanin1_expr = get_boolean_expression(gate['fanin1'])
            fanin2_expr = get_boolean_expression(gate['fanin2'])
            return f'({fanin1_expr} xor {fanin2_expr})'
        elif gate['type'] == 'xnor':
            fanin1_expr = get_boolean_expression(gate['fanin1'])
            fanin2_expr = get_boolean_expression(gate['fanin2'])
            return f'({fanin1_expr} xnor {fanin2_expr})'
        elif gate['type'] == 'not':
            fanin1_expr = get_boolean_expression(gate['fanin1'])
            return f'(not {fanin1_expr})'

    output_gate = max(gates.keys(), default=None)
    if output_gate is None or gates[output_gate]['type'] == 'from':
        return ''  # Skip generating equation if output gate is 'from' or if there are no gates
    else:
        return get_boolean_expression(output_gate)

def dfs(gate_num):
    print("Processing gate number:", gate_num)  # Debug print
    try:
        gate = gates[gate_num]  # No need to adjust gate_num here
    except KeyError:
        print("KeyError: Gate number", gate_num, "not found in gates dictionary")
        return None  # Handle KeyError gracefully
    if gate['type'] == 'inpt':
        return {'CC0': 1, 'CC1': 1, 'CO': gate_num}
    elif gate['type'] == 'from':
        return dfs(gate['fanin'])
    elif gate['type'] in ['and', 'or', 'nand', 'nor', 'xor', 'xnor']:
        fanin1_result = dfs(gate['fanin1'])
        fanin2_result = dfs(gate['fanin2'])
        # Combine results based on gate type
        # (Not implemented in the provided code snippet)
        return {}  # Placeholder for the combined result


# Read netlist from a .txt file
file_path = r'E:\NIRMA STUDY MATERIAL\SEM 6\TVDC\SPECIAL ASSIGNMENT\fwd\netlist_test1.txt'  # Update with the path to your .txt file
with open(file_path, 'r') as file:
    netlist_txt = file.read()
gates = parse_netlist(netlist_txt)

boolean_equation = generate_boolean_equation(gates)
print("Boolean Equation is:")
print(boolean_equation)

# Perform controllability analysis
controllability_result = controllability_analysis(gates)
print("\nControllability Analysis:")
for gate_num, result in controllability_result.items():
    print(f"Gate {gate_num}: {result}")

from schemdraw.parsing import logicparse
logicparse(boolean_equation, outlabel='PO')


Boolean Equation is:
((((x1 and x2) or (x3 xor x4)) nor ((not x5) and (not (x3 xor x4)))) xnor (((x1 and x2) or (x3 xor x4)) nor ((not x5) and (not (x3 xor x4)))))


NameError: name 'controllability_analysis' is not defined